# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, neighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# 2. Load Dataset

In [2]:
iris = datasets.load_iris()
# iris

In [3]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [4]:
features_array = iris.data
target_array = iris.target

# 3. Standardization

In [5]:
standardizer = StandardScaler()

In [6]:
standardized_features_array = standardizer.fit_transform(features_array)

In [7]:
print(round(standardized_features_array.mean()))
print(round(standardized_features_array.std()))

0
1


# 4. Create Model Object

In [8]:
model = neighbors.NearestNeighbors(n_neighbors=2)

# 5. Check Nearest Neighbors

In [9]:
model.fit(standardized_features_array)

NearestNeighbors(n_neighbors=2)

In [10]:
new_sample = [1, 1, 1, 1]

In [11]:
distance, indices = model.kneighbors([new_sample])

![](https://www.saedsayad.com/images/KNN_similarity.png)m

In [12]:
print(distance)
print(indices)

[[0.49140089 0.74294782]]
[[124 110]]


In [13]:
features_array[indices]

array([[[6.7, 3.3, 5.7, 2.1],
        [6.5, 3.2, 5.1, 2. ]]])

In [14]:
standardized_features_array[indices]

array([[[1.03800476, 0.55861082, 1.10378283, 1.18556721],
        [0.79566902, 0.32841405, 0.76275827, 1.05393502]]])

# 6. Create Model obejct(KNN)

In [15]:
model = neighbors.KNeighborsClassifier(n_neighbors=5, n_jobs=-1, metric='euclidean', weights='distance')

In [16]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(standardized_features_array,
                                                                    target_array,
                                                                    test_size=0.3,
                                                                    random_state=42)

In [17]:
model.fit(x_train, y_train)

KNeighborsClassifier(metric='euclidean', n_jobs=-1, weights='distance')

# 7. Test Model object

In [18]:
model.predict(x_test)

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0,
       0])

In [19]:
# accuracy_score
accuracy_score(model.predict(x_test), y_test)

1.0

# 8. Searching for Proper K

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [21]:
model

KNeighborsClassifier(metric='euclidean', n_jobs=-1, weights='distance')

In [22]:
# Create Pipeline
pipe = Pipeline([('standardizer',standardizer),('knn',model)])
pipe

Pipeline(steps=[('standardizer', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(metric='euclidean', n_jobs=-1,
                                      weights='distance'))])

In [23]:
# candidate for GridSearching
search_space = [{'knn__n_neighbors':range(1, 11)}]

In [24]:
# create GridSearch
grid_model = GridSearchCV(
    estimator=pipe,
    param_grid=search_space,
    cv=5,
    n_jobs=-1
)

grid_model

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardizer', StandardScaler()),
                                       ('knn',
                                        KNeighborsClassifier(metric='euclidean',
                                                             n_jobs=-1,
                                                             weights='distance'))]),
             n_jobs=-1, param_grid=[{'knn__n_neighbors': range(1, 11)}])

In [25]:
grid_model.fit(features_array, target_array)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardizer', StandardScaler()),
                                       ('knn',
                                        KNeighborsClassifier(metric='euclidean',
                                                             n_jobs=-1,
                                                             weights='distance'))]),
             n_jobs=-1, param_grid=[{'knn__n_neighbors': range(1, 11)}])

In [26]:
grid_model.best_estimator_.get_params()['knn__n_neighbors']

6